# GMM-based Color Image Segmentation
利用 Gaussian Mixture Model 完成 color image segmentation

## Scenario 1
以 Soccer1.jpg 中的場地 pixel 建構 GMM，稱此模型為 M1，並以 soccer1.jpg 做測試

In [16]:
import cv2
import numpy as np
from sklearn.mixture import GaussianMixture
import csv
import pathlib

In [17]:
def write_file(dir_name, file ,file_name):
    p = pathlib.Path(dir_name)
    p.mkdir(parents=True, exist_ok=True)
    filepath = dir_name + '/' + file_name
    cv2.imwrite(filepath, file)

In [18]:
def segment_filter(dir_name, level, thres, probs, rows, cols):
    segmented_img = probs[:, level] < thres
    segmented_img = segmented_img.reshape(rows, cols).astype(np.uint8)
    segmented_255_img = segmented_img * 255

    file_name = str(level) + '_' + str(thres)+ '.jpg'
    write_file(dir_name, segmented_255_img, file_name)
    return segmented_img

In [19]:
def evaluation(segmented_img, gr_list):
    cmp_seg_list = list(segmented_img.flatten())
    same_num = 0
    for i in range(len(gr_list)):
            if gr_list[i] == cmp_seg_list[i]:
                    same_num += 1
    return same_num / len(gr_list)     

In [20]:
def read_gr(file_name):
    file = open(file_name)
    csvreader = csv.reader(file)
    next(csvreader)
    gr_list = []
    for row in csvreader:
            gr_list.append(int(row[2]))
    return gr_list

In [21]:
img = cv2.imread('soccer1.jpg')
rows, cols, channels = img.shape
pixel_values = img.reshape(rows * cols, channels)

In [22]:
n_components = 3  # 設定Gaussian混合模型的成分數
M1 = GaussianMixture(n_components=n_components, covariance_type='full')
M1.fit(pixel_values)

GaussianMixture(n_components=3)

In [23]:
gr_list = read_gr('soccer1_mask.csv')
dir_name = 'S1'
# 調整參數
para_list = []
for level in range(n_components):
        for thres in range(1, 10, 1):
                thres = thres / 10
                # 進行分割
                probs = M1.predict_proba(pixel_values)
                segmented_img = segment_filter(dir_name, level, thres, probs, rows, cols)
                # 算出 precision
                precision = evaluation(segmented_img, gr_list)
                
                para_list.append([level, thres, precision])

max_index = np.argmax(np.array(para_list)[:,2])
score = para_list[max_index][2]
para_level = para_list[max_index][0]
para_threshold = para_list[max_index][1]
print('S1 best precision score:{:.2f} and its level is {}, threshold is {}.'.format(score, para_level, para_threshold))

S1 best precision score:0.93 and its level is 1, threshold is 0.9.


## Scenario 2
以 M1 針對 soccer2.jpg 做測試

In [24]:
img = cv2.imread('soccer2.jpg')
rows, cols, channels = img.shape
pixel_values = img.reshape(rows * cols, channels)

In [25]:
gr_list = read_gr('soccer2_mask.csv')
dir_name = 'S2'

# 進行分割
probs = M1.predict_proba(pixel_values)
segmented_img = segment_filter(dir_name, para_level, para_threshold, probs, rows, cols)
# 算出 precision
precision = evaluation(segmented_img, gr_list)

print('S2 precision:{:.2f} '.format(precision))

S2 precision:0.75 


## Scenario 3
以 soccer1.jpg 與 soccer2.jpg 中的場地 pixel 建構 GMM，稱此模型為 M2，並以 soccer1.jpg 以及 soccer2.jpg 做測試

In [26]:
img1 = cv2.imread('soccer1.jpg')
img2 = cv2.imread('soccer2.jpg')
rows, cols, channels = img.shape
pixel_values = img.reshape(rows * cols, channels)

combined_img = np.concatenate((img1, img2), axis=0)

pixel_values = combined_img.reshape((-1, 3))

n_components = 3  # 設定為兩個 label：背景和前景
M2 = GaussianMixture(n_components=n_components)
M2.fit(pixel_values)


gr_list1 = read_gr('soccer1_mask.csv')
gr_list2 = read_gr('soccer2_mask.csv')
gr_list = gr_list1 + gr_list2
dir_name = 'S3'

# 預測像素屬於哪個 label
probs = M2.predict_proba(pixel_values)

para_list = []
for level in range(n_components):
        for thres in range(1, 10, 1):
                thres = thres / 10
                # 進行分割
                probs = M2.predict_proba(pixel_values)
                segmented_img = probs[:, level] < thres
                segmented_img = segmented_img.reshape(combined_img.shape[:2])
                print(segmented_img.shape)
                seg_img1 = segmented_img[:img1.shape[0], :].astype(np.uint8)
                seg_img2 = segmented_img[img1.shape[0]:, :].astype(np.uint8)
                print(seg_img1.shape)
                break
                segmented_255_img1 = seg_img1 * 255
                segmented_255_img2 = seg_img2 * 255

                file1_name = str(level) + '_' + str(thres)+ '_1.jpg'
                file2_name = str(level) + '_' + str(thres)+ '_2.jpg'
                write_file(dir_name, segmented_255_img1, file1_name)
                write_file(dir_name, segmented_255_img2, file2_name)
                # 算出 precision
                precision = evaluation(segmented_img, gr_list)
                
                para_list.append([level, thres, precision])

max_index = np.argmax(np.array(para_list)[:,2])
score = para_list[max_index][2]
para_level = para_list[max_index][0]
para_threshold = para_list[max_index][1]
print('S3 best precision score:{:.2f} and its level is {}, threshold is {}.'.format(score, para_level, para_threshold))


# 分離兩張圖片的分割結果
seg_img1 = segmented_img[:img1.shape[0], :]
seg_img2 = segmented_img[img1.shape[0]:, :]

S3 best precision score:0.93 and its level is 2, threshold is 0.1.
